# Building the Boston Crime Database

This notebook describes the process adopted to build the Boston Crime 
database from a CSV file 

In [25]:
import psycopg2
# Connect to dq database and create new database

conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE DATABASE crime_db;")
conn.commit()
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cursor = conn.cursor()
cursor.execute("CREATE SCHEMA crimes;")

ProgrammingError: database "crime_db" already exists


In [6]:
import csv

with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    
    first_row = next(reader)
    
    print(col_headers)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [4]:
# Read a csv file and column index and return unique values of the column 
import csv
def get_col_value_set(csv_file, col_index):
    col_values = set()
    with open(csv_file) as f:
        reader = csv.reader(f, delimiter=",", skipinitialspace=True)
        next(reader) #skip the header
        for row in reader:
            col_values.add(row[col_index])
        return col_values

for x in range(7):
    unique_col_values = get_col_value_set('boston.csv', x)
    print("Column", x, "has", len(unique_col_values), "unique values:")
#     print(unique_col_values)
        

Column 0 has 298329 unique values:
Column 1 has 219 unique values:
Column 2 has 239 unique values:
Column 3 has 1177 unique values:
Column 4 has 7 unique values:
Column 5 has 18177 unique values:
Column 6 has 18177 unique values:


In [22]:
# Identify what index the description column is
print(col_headers)

# Determine the maximum length of a given column
col_values = get_col_value_set('boston.csv', 6)

max_len = 0
max_val = ''
for val in col_values:
    length = len(val)
    if length > max_len:
        max_len = length
        max_val = val
print("Max length:", max_len)
print("Value with maximum length:", max_val)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
Max length: 12
Value with maximum length: -71.05260013


Based on the analysis of each column value from the CSV file, in creating the database tables, 
- incident_number will be integer
- offense_code will be varchar(4)
- description will be varchar(58)
- date will be date
- day_of_the_week will be an ENUM
- lat will be numeric
- long will be numeric

In [29]:
# Creating an ENUM type for day_of_the_week
import psycopg2

conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE TYPE weekday AS ENUM('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday');")
conn.commit()
conn.close()

ProgrammingError: type "weekday" already exists


In [34]:
# Creating the crimes.boston_crimes table

conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cursor = conn.cursor()
# cursor.execute("CREATE SCHEMA crimes;")
cursor.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number integer PRIMARY KEY,
        offense_code VARCHAR(4),
        description VARCHAR(58),
        incidence_date DATE,
        day_of_the_week weekday,
        lat numeric,
        long numeric
    );
""")
conn.commit()
conn.close()

In [35]:
# Load data from boston.csv file into the boston_crimes table

conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cursor = conn.cursor()

with open("boston.csv") as f:
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    conn.commit()
    conn.close()

In [36]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cursor = conn.cursor()

# Revoke all privileges on the crimes_db by the public group 
cursor.execute("REVOKE ALL ON SCHEMA public FROM public;")
cursor.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

# Create two user groups
cursor.execute("CREATE GROUP readonly NOLOGIN;")
cursor.execute("CREATE GROUP readwrite NOLOGIN;")

# Grant CONNECT privilege to both user groups
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

# Grant USAGE of crimes schema to both user groups
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Grant group specific privileges to the user groups
cursor.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cursor.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

# Create user called data_analyst and assign the user to readonly group
cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cursor.execute("GRANT readonly TO data_analyst;")

# Create user called data_scientist and assign the user to readwrite group
cursor.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cursor.execute("GRANT readwrite TO data_scientist;")

conn.commit()
conn.close()

In [39]:
# Test that table set up is accurate
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cursor = conn.cursor()

cursor.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee='readwrite';")
cursor.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [40]:
cursor.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee='readonly';")
cursor.fetchall()

[('readonly', 'SELECT')]